# This code goes through a file of recordings to generate a list of dates they were collected, then asks the user for the surgery date and saves it.
## Run this cell if you'd like to run the code for each bird's folder individually:
### If you want to use a pop-up file navigator to select the folder, write: file_path = [] in line 8
### If you'd like to copy the file path into the code, adjust line 8

In [ ]:
from pathlib import Path
import time
import json
from tkinter import Tk
from tkinter import filedialog
from datetime import datetime

# Function to validate user input for treatment date
def get_treatment_date():
    while True:
        treatment_date_input = input("Please enter the treatment date (YYYY-MM-DD): ")
        try:
            # Parse the input date to check if it's valid
            treatment_date = datetime.strptime(treatment_date_input, '%Y-%m-%d')
            return treatment_date.strftime('%Y-%m-%d')  # Return the date in the proper format
        except ValueError:
            print("Invalid date format. Please try again.")

# Function to get treatment type
def get_treatment_type():
    treatment_type = input("Please enter the treatment type: ")
    return treatment_type

# Get the treatment date and treatment type from the user
treatment_date = get_treatment_date()
treatment_type = get_treatment_type()

# Initialize the folder_path variable
folder_path = []

# If folder_path is empty, open a GUI to let the user select a folder
if not folder_path:
    # Hide the main Tkinter window
    Tk().withdraw()
    
    # Open a dialog for folder selection
    folder_path = filedialog.askdirectory(title="Select Main Folder")

# Check if a folder was selected
if folder_path:
    # Path object for the selected folder
    main_folder = Path(folder_path)

    # Dictionary to store subdirectory info
    folder_data = {
        "treatment_date": treatment_date,  # Store the treatment date entered by the user
        "treatment_type": treatment_type,  # Store the treatment type entered by the user
        "subdirectories": {}
    }

    # Loop through each subdirectory in the main folder
    for subdirectory in main_folder.iterdir():
        if subdirectory.is_dir():  # Ensure it is a subdirectory
            # Get the creation (birth) date of the subdirectory
            subdirectory_creation_time = subdirectory.stat().st_birthtime
            subdirectory_creation_date = time.strftime('%Y-%m-%d', time.localtime(subdirectory_creation_time))

            # Set to store unique file creation (birth) dates within the subdirectory
            unique_birth_dates = set()

            # Loop through files within this subdirectory
            for file in subdirectory.rglob('*'):
                if file.is_file():
                    # Get the creation (birth) time of the file
                    file_creation_time = file.stat().st_birthtime
                    
                    # Convert the timestamp to a date (not including time)
                    file_creation_date = time.strftime('%Y-%m-%d', time.localtime(file_creation_time))
                    
                    # Add the file creation date to the set
                    unique_birth_dates.add(file_creation_date)
            
            # Add the subdirectory creation date and file creation dates to the dictionary
            folder_data["subdirectories"][subdirectory.name] = {
                "subdirectory_creation_date": subdirectory_creation_date,
                "unique_file_creation_dates": sorted(list(unique_birth_dates))  # Convert set to a sorted list
            }

    # Save the data to a .json file with the main folder's name
    json_file_name = f"{main_folder.name}_creation_data.json"
    json_file_path = main_folder / json_file_name

    with open(json_file_path, 'w') as json_file:
        json.dump(folder_data, json_file, indent=4)

    print(f"Data has been saved to {json_file_path}")
else:
    print("No folder was selected.")


## Run this cell if you'd like the code to automatically loop through all the birds' folders from a main directory. You will need to enter the treatment/surgery date and treatment/surgery type (e.g. Area X lesion) for each bird.
### If you want to use a pop-up file navigator to select the folder, write: file_path = [] in line 8
### If you'd like to copy the file path into the code, adjust line 8

In [ ]:
from pathlib import Path
import time
import json
from tkinter import Tk
from tkinter import filedialog
from datetime import datetime

# Function to validate user input for treatment date
def get_treatment_date(folder_name):
    while True:
        treatment_date_input = input(f"Please enter the treatment date for {folder_name} (YYYY-MM-DD): ")
        try:
            # Parse the input date to check if it's valid
            treatment_date = datetime.strptime(treatment_date_input, '%Y-%m-%d')
            return treatment_date.strftime('%Y-%m-%d')  # Return the date in the proper format
        except ValueError:
            print("Invalid date format. Please try again.")

# Function to get treatment type
def get_treatment_type(folder_name):
    treatment_type = input(f"Please enter the treatment type for {folder_name}: ")
    return treatment_type

# Initialize the folder_path variable
folder_path = "/Volumes/ROSE2-SSD/RHV_song_counts/untitled folder"

# If folder_path is empty, open a GUI to let the user select a folder
if not folder_path:
    # Hide the main Tkinter window
    Tk().withdraw()
    
    # Open a dialog for selecting the directory containing multiple main folders
    folder_path = filedialog.askdirectory(title="Select Parent Directory with Multiple Main Folders")

# Check if a folder was selected
if folder_path:
    # Path object for the selected parent directory
    parent_directory = Path(folder_path)

    # Iterate over each main folder in the parent directory
    for main_folder in parent_directory.iterdir():
        if main_folder.is_dir():  # Ensure it's a directory (main folder)

            # Prompt for treatment date and treatment type for each main folder
            print(f"\nProcessing folder: {main_folder.name}")
            treatment_date = get_treatment_date(main_folder.name)
            treatment_type = get_treatment_type(main_folder.name)

            # Dictionary to store subdirectory info for the current main folder
            folder_data = {
                "treatment_date": treatment_date,  # Store the treatment date entered by the user
                "treatment_type": treatment_type,  # Store the treatment type entered by the user
                "subdirectories": {}
            }

            # Loop through each subdirectory in the main folder
            for subdirectory in main_folder.iterdir():
                if subdirectory.is_dir():  # Ensure it is a subdirectory
                    # Get the creation (birth) date of the subdirectory

                    # Use this if running on a Unix computer (e.g. on the lambda compuu)
                    subdirectory_creation_time = subdirectory.stat().st_mtime
                    #Use this if running on a Windows or Mac computer:
                    #subdirectory_creation_time = subdirectory.stat().st_birthtime
                    
                    subdirectory_creation_date = time.strftime('%Y-%m-%d', time.localtime(subdirectory_creation_time))

                    # Set to store unique file creation (birth) dates within the subdirectory
                    unique_birth_dates = set()

                    # Loop through files within this subdirectory
                    for file in subdirectory.rglob('*'):
                        if file.is_file():
                            
                            # Get the creation (birth) time of the file
                            #use if running on a Unix computer:
                            file_creation_time = file.stat().st_mtime
                            #Use in Windows or Mac computer:
                            #file_creation_time = file.stat().st_birthtime
                            
                            # Convert the timestamp to a date (not including time)
                            file_creation_date = time.strftime('%Y-%m-%d', time.localtime(file_creation_time))
                            
                            # Add the file creation date to the set
                            unique_birth_dates.add(file_creation_date)
                    
                    # Add the subdirectory creation date and file creation dates to the dictionary
                    folder_data["subdirectories"][subdirectory.name] = {
                        "subdirectory_creation_date": subdirectory_creation_date,
                        "unique_file_creation_dates": sorted(list(unique_birth_dates))  # Convert set to a sorted list
                    }

            # Save the data to a .json file with the main folder's name
            json_file_name = f"{main_folder.name}_creation_data.json"
            json_file_path = main_folder / json_file_name

            with open(json_file_path, 'w') as json_file:
                json.dump(folder_data, json_file, indent=4)

            print(f"Data for {main_folder.name} has been saved to {json_file_path}")

else:
    print("No folder was selected.")


Processing folder: 205


Invalid date format. Please try again.
Invalid date format. Please try again.


KeyboardInterrupt: Interrupted by user